In [1]:
#imports
import base64
import sys
import time
import pyvisa
from time import sleep
from toptica.lasersdk.client import Client, SerialConnection, UserLevel

def find_toptica_port():
    rm = pyvisa.ResourceManager("@py")
    for resource in rm.list_resources():
        if resource[0:4]=="ASRL":
            try:
                dlc_connection_port = "COM"+resource[4]
                #set up connection with DLC smart and check user level
                with Client(SerialConnection(dlc_connection_port)) as client:
                    print("Found Toptica on "+dlc_connection_port)
                    user_level = client.get('ul')
                    print(client.get('general:serial-number'))
                    dlc_smart = client.get('general:system-type')
                    print(f"Connected to: {dlc_smart}")
                    print(f"Current User Level: {user_level}")
                    return dlc_connection_port
            except:
                continue
    rm.close()



In [3]:
rm = pyvisa.ResourceManager("@py")
rm.list_resources()

('ASRL4::INSTR', 'ASRL3::INSTR', 'ASRL9::INSTR', 'TCPIP::192.168.0.196::INSTR')

In [25]:
#set connections to DLC smart and lock-in amplifier
lockin_address = 'GPIB::8::INSTR'
dlc_connection_port = find_toptica_port()
#lock-in sensitivity setting
sensitivity_setting = 15
#set frequency to measure at
set_frequency = 120 #GHz
time_constant = 6 #lock-in time constant setting, specifics found in SR830 manual
low_pass_filter = 1 #lock-in low pass filter slope setting, specifics found in SR830 manual

Found Toptica on COM9
DLC smart_015174
Connected to: DLCsmartTHz
Current User Level: 3


In [18]:
#set up connection with lock-in amplifier
# Make sure the USB is plugged in before the lockin is turned on!
rm = pyvisa.ResourceManager()
lockin = rm.open_resource(lockin_address)
idn = lockin.query('*IDN?')
print(f'Connected to: {idn}')

Connected to: Stanford_Research_Systems,SR830,s/n46458,ver1.07 



In [19]:
#set up connection with DLC smart and check user level
with Client(SerialConnection(dlc_connection_port)) as client:
    user_level = client.get('ul')
    print(client.get('general:serial-number'))
    dlc_smart = client.get('general:system-type')
    print(f"Connected to: {dlc_smart}")
    print(f"Current User Level: {user_level}")

DLC smart_015174
Connected to: DLCsmartTHz
Current User Level: 3


In [20]:
#turn on lasers (check emission lights)
#if lasers do not power on, check the frontkey and user level
with Client(SerialConnection(dlc_connection_port)) as client:
    client.set('laser-operation:emission-global-enable', True)
    laser_on_status = client.get('laser-operation:emission-global-enable')
    print(f'Lasers on: {laser_on_status}')
#wait 5 secs for lasers to adjust
time.sleep(1)

Lasers on: True


In [22]:
#set DLC Smart bias amplitude and offset to default
with Client(SerialConnection(dlc_connection_port)) as client:
    client.set('lockin:mod-out-amplitude', 2.0)
    client.set('lockin:mod-out-offset', 0.0)
#set DLC Smart bias frequency to 1000
    client.set('lockin:frequency', 1000)
    print(f"Current DLC Smart Bias Amplitude (V): {client.get('lockin:mod-out-amplitude')}")
    print(f"Current DLC Smart Bias Offset (V): {client.get('lockin:mod-out-offset')}")
    print(f"Current DLC Smart Bias Frequency (Hz): {client.get('lockin:frequency')}")
#wait 5 secs for DLC smart to adjust
time.sleep(5)

Current DLC Smart Bias Amplitude (V): 1.999969482421875
Current DLC Smart Bias Offset (V): 0
Current DLC Smart Bias Frequency (Hz): 1000


In [26]:
#set DLC Smart frequency to value you wish to measure
with Client(SerialConnection(dlc_connection_port)) as client:
    client.set('frequency:frequency-set', set_frequency)
    print(f"Current DLC Smart Frequency (GHz): {client.get('frequency:frequency-act')}")

Current DLC Smart Frequency (GHz): 100.00343561015325


In [28]:
#set lock-in amplifier sensitivity
lockin.write(f'SENS {sensitivity_setting}')
lockin_sensitivity = lockin.query('SENS?')
print(f"Current Lock-in Sensitivity setting: {lockin_sensitivity}")
#set lock-in time constant setting
lockin.write(f'OFLT {time_constant}')
print(f"Current Time Constant: {lockin.query('OFLT?')}")
#set lock-in low pass filter slope
lockin.write(f'OFSL {low_pass_filter}')
print(f"Current Low Pass Filter: {lockin.query('OFSL?')}")

Current Lock-in Sensitivity setting: 15

Current Time Constant: 6

Current Low Pass Filter: 1



In [27]:
#set lock-in amplifier sensitivity
#lockin.write(f'SENS {sensitivity_setting}')
lockin_sensitivity = lockin.query('SENS?')
print(f"Current Lock-in Sensitivity setting: {lockin_sensitivity}")
#set lock-in time constant setting
#lockin.write(f'OFLT {time_constant}')
print(f"Current Time Constant: {lockin.query('OFLT?')}")
#set lock-in low pass filter slope
#lockin.write(f'OFSL {low_pass_filter}')
print(f"Current Low Pass Filter: {lockin.query('OFSL?')}")

Current Lock-in Sensitivity setting: 9

Current Time Constant: 6

Current Low Pass Filter: 1

